In [17]:
# Ziel: Für jede eindeutige convoID (excludeGeneral=0) prüfen, wie viele deutsche Synonyme es für den zugehörigen ICD-Titel gibt,
# und ein DataFrame mit convoID, icd_title und syn_count erstellen.

from pymongo import MongoClient
import pandas as pd

# 1. Verbindungen und Collections
client    = MongoClient("mongodb://localhost:27018/")
col_trans = client["transcriptions"]["transcripts_denis"]
col_mimic = client["MIMIC-IV"]["NLP-EXPANDED-prammed-postprocessed_translation"]
col_syn   = client["En2DeSyn"]["ED_deSynonyms"]

# 2. Einzigartige convoIDs mit excludeGeneral=0 holen
convo_ids = col_trans.distinct("convoID", {"excludeGeneral": 0})
df_trans  = pd.DataFrame(convo_ids, columns=["convoID"])

# 3. MIMIC-ICD-Titel pro stay_id (=convoID) ziehen
mimic_docs = col_mimic.find(
    {"stay_id": {"$in": convo_ids}},
    {"stay_id": 1, "json_data_used.ED-Diagnosis-prammed.icd_title": 1, "_id": 0}
)
rows = []
for d in mimic_docs:
    rows.append({
        "convoID": d["stay_id"],
        "icd_title": d.get("json_data_used", {})
                      .get("ED-Diagnosis-prammed", {})
                      .get("icd_title")
    })
df_mimic = pd.DataFrame(rows)

# 4. Synonymenzähler vorbereiten
syn_docs = col_syn.find({}, {"icd_title": 1, "de_synonyms": 1, "_id": 0})
df_syn   = pd.DataFrame(syn_docs)
df_syn["syn_count"] = df_syn["de_synonyms"].apply(lambda x: len(x) if isinstance(x, list) else 0)

# 5. Zusammenführen: trans → mimic → syn
df = (
    df_trans
    .merge(df_mimic, on="convoID", how="left")
    .merge(df_syn[["icd_title", "syn_count"]], on="icd_title", how="left")
)
df["syn_count"] = df["syn_count"].fillna(0).astype(int)

# 6. Überblick ausgeben
print(f"Einzigartige convoIDs (excludeGeneral=0): {len(df_trans)}")
print(f"ConvoIDs ohne ICD-Titel: {(df['icd_title'].isna()).sum()}")
print(f"ConvoIDs ohne Synonyme: {(df['syn_count']==0).sum()}")

# 7. Als CSV speichern (optional)
df.to_csv("convo_synonyms_overview.csv", index=False)
print("Ergebnis in 'convo_synonyms_overview.csv' gespeichert.")



Einzigartige convoIDs (excludeGeneral=0): 99
ConvoIDs ohne ICD-Titel: 0
ConvoIDs ohne Synonyme: 39
Ergebnis in 'convo_synonyms_overview.csv' gespeichert.


In [18]:
# Ziel: In der Collection `ED_deSynonyms` ermitteln, wie viele `icd_title`-Einträge doppelt vorkommen,
# und nach Entfernen der Duplikate (pro Titel nur ein Dokument) die verbleibende Anzahl im Vergleich
# zur aktuellen Gesamtanzahl der Dokumente anzeigen.

from pymongo import MongoClient
import pandas as pd

# MongoDB-Verbindung
client = MongoClient("mongodb://localhost:27018/")
col_syn = client["En2DeSyn"]["ED_deSynonyms"]

# 1. Gesamtzahl der Dokumente in der Collection
total_docs = col_syn.count_documents({})

# 2. Gruppieren nach icd_title und zählen
pipeline = [
    {"$group": {"_id": "$icd_title", "count": {"$sum": 1}}},
    {"$sort": {"count": -1}}
]
grouped = list(col_syn.aggregate(pipeline))
df_grouped = pd.DataFrame(grouped).rename(columns={"_id": "icd_title"})

# 3. Duplikate herausfiltern (count > 1)
df_dups = df_grouped[df_grouped["count"] > 1]
duplicate_titles_count = len(df_dups)  # Anzahl verschiedener Titel, die mehrfach vorkommen
duplicate_entries_count = df_dups["count"].sum() - duplicate_titles_count  # Überschüssige Dokumente

# 4. Einzigartige Titel-Anzahl (nach Entfernen aller Duplikate)
unique_titles_count = len(df_grouped)

# 5. Ergebnisse ausgeben
print(f"Gesamt-Dokumente in ED_deSynonyms:        {total_docs}")
print(f"Einzigartige ICD-Titel:                   {unique_titles_count}")
print(f"Titel mit Duplikaten:                     {duplicate_titles_count}")
print(f"Überschüssige Einträge durch Duplikate:   {duplicate_entries_count}")

# 6. Optional: DataFrame der doppelten Titel anzeigen
df_dups.reset_index(drop=True, inplace=True)
df_dups.head(10)  # zeigt die ersten 10 doppelten Titel mit ihren counts



Gesamt-Dokumente in ED_deSynonyms:        5662
Einzigartige ICD-Titel:                   5662
Titel mit Duplikaten:                     0
Überschüssige Einträge durch Duplikate:   0


,icd_title,count


In [ ]:
# Ziel: Duplikate in ED_deSynonyms entfernen, dabei Dokumente mit deutschen Synonymen bevorzugt behalten.

from pymongo import MongoClient
from bson import ObjectId

# 1. Verbindung und Collection
client = MongoClient("mongodb://localhost:27018/")
col = client["En2DeSyn"]["ED_deSynonyms"]

# 2. Alle ICD-Titel ermitteln, die mehrfach vorkommen, mit ihren Dokument-IDs und Synonym-Anzahlen
pipeline = [
    {"$group": {
        "_id": "$icd_title",
        "ids": {"$push": "$_id"},
        "syn_counts": {"$push": {"$size": {"$ifNull": ["$de_synonyms", []]}}},
        "count": {"$sum": 1}
    }},
    {"$match": {"count": {"$gt": 1}}}
]
duplicates = list(col.aggregate(pipeline))

# 3. Für jede Gruppe die zu löschenden IDs sammeln
to_delete = []
for group in duplicates:
    # IDs und ihre jeweilige Anzahl an de_synonyms
    id_syn_pairs = list(zip(group["ids"], group["syn_counts"]))
    # Zuerst diejenigen mit Synonymen (syn_count > 0), dann ohne
    # Sortiere absteigend nach syn_count, so dass Dokumente mit den meisten Synonymen zuerst kommen
    id_syn_pairs.sort(key=lambda pair: pair[1], reverse=True)
    # Behalte das erste Dokument, lösche alle anderen
    keep_id = id_syn_pairs[0][0]
    delete_ids = [doc_id for doc_id, syn_count in id_syn_pairs[1:]]
    to_delete.extend(delete_ids)

# 4. Löschen der gesammelten Dokumente
if to_delete:
    result = col.delete_many({"_id": {"$in": to_delete}})
    print(f"Anzahl gelöschter Dokumente: {result.deleted_count}")
else:
    print("Keine Duplikate zum Löschen gefunden.")



In [20]:
# Ziel: Anzahl der Einträge in `ED_deSynonyms` ermitteln, die keine deutschen Synonyme haben

from pymongo import MongoClient

# Verbindung zur MongoDB und Auswahl der Collection
client = MongoClient("mongodb://localhost:27018/")
col = client["En2DeSyn"]["ED_deSynonyms"]

# Zähle Dokumente, bei denen `de_synonyms` leer ist oder nicht existiert
count_no_syn = col.count_documents({
    "$or": [
        {"de_synonyms": {"$exists": False}},
        {"de_synonyms": {"$size": 0}}
    ]
})

print(f"Einträge ohne deutsche Synonyme: {count_no_syn}")


Einträge ohne deutsche Synonyme: 0


In [21]:
# Ziel: Für jede einzigartige convoID prüfen, welcher ICD-Titel zugeordnet ist und wie viele deutsche Synonyme dafür vorliegen.

from pymongo import MongoClient
import pandas as pd
from IPython.display import display

# 1. MongoDB-Verbindung und Collections
client    = MongoClient("mongodb://localhost:27018/")
col_trans = client["transcriptions"]["transcripts_denis"]
col_mimic = client["MIMIC-IV"]["NLP-EXPANDED-prammed-postprocessed_translation"]
col_syn   = client["En2DeSyn"]["ED_deSynonyms"]

# 2. Einzigartige convoIDs mit excludeGeneral=0
convo_ids = col_trans.distinct("convoID", {"excludeGeneral": 0})
df_trans  = pd.DataFrame(convo_ids, columns=["convoID"])

# 3. ICD-Titel aus MIMIC für diese convoIDs
mimic_cursor = col_mimic.find(
    {"stay_id": {"$in": convo_ids}},
    {"stay_id": 1, "json_data_used.ED-Diagnosis-prammed.icd_title": 1, "_id": 0}
)
rows = []
for doc in mimic_cursor:
    icd = (doc.get("json_data_used", {})
              .get("ED-Diagnosis-prammed", {})
              .get("icd_title"))
    rows.append({"convoID": doc["stay_id"], "icd_title": icd})
df_mimic = pd.DataFrame(rows)

# 4. Deutsche Synonyme und Zähler aus ED_deSynonyms
icd_list = df_mimic["icd_title"].dropna().unique().tolist()
syn_cursor = col_syn.find(
    {"icd_title": {"$in": icd_list}},
    {"icd_title": 1, "de_synonyms": 1, "_id": 0}
)
syn_rows = []
for doc in syn_cursor:
    syns = doc.get("de_synonyms") or []
    syn_rows.append({
        "icd_title": doc["icd_title"],
        "synonyms": syns,
        "syn_count": len(syns)
    })
df_syn = pd.DataFrame(syn_rows)

# 5. Zusammenführen und fehlende Einträge auffüllen
df = (
    df_trans
    .merge(df_mimic, on="convoID", how="left")
    .merge(df_syn, on="icd_title", how="left")
)
df["synonyms"] = df["synonyms"].apply(lambda x: x if isinstance(x, list) else [])
df["syn_count"] = df["syn_count"].fillna(0).astype(int)

# 6. Ergebnis anzeigen
display(df)

# Überblick
print(f"Anzahl eindeutiger convoIDs: {len(df)}")
print(f"davon mit ≥1 deutschem Synonym: {(df['syn_count'] > 0).sum()}")
print(f"davon ohne Synonyme:           {(df['syn_count'] == 0).sum()}")


,convoID,icd_title,synonyms,syn_count
0,30013825,"Oth fracture of head and neck of right femur, ...",[],0
1,30114157,Oth intartic fracture of lower end of left rad...,[],0
2,30238398,"Foreign body in left ear, initial encounter",[],0
3,30328724,Nausea,"[an uebelkeit leiden, an uebelkeit leidend, na...",8
4,30394981,MELENA,"[blut im stuhl, blutiger stuhl, blutstuhl]",3
...,...,...,...,...
94,39609911,"Other psychoactive substance abuse, uncomplicated",[],0
95,39637559,ABDOMINAL PAIN EPIGASTRIC,"[epigastralgie, epigastrischer schmerz, epigas...",4
96,39639914,DIAB HYPEROSM COMA IDDM,[],0
97,39699150,LONG-TERM (CURRENT) USE OF INSULIN,[],0


Anzahl eindeutiger convoIDs: 99
davon mit ≥1 deutschem Synonym: 60
davon ohne Synonyme:           39


In [22]:
# Ziel: Lade die convoID-/ICD-/Synonym-Daten, filtere Einträge ohne deutsche Synonyme und speichere sie als CSV

from pymongo import MongoClient
import pandas as pd

# MongoDB-Verbindung
client    = MongoClient("mongodb://localhost:27018/")
col_trans = client["transcriptions"]["transcripts_denis"]
col_mimic = client["MIMIC-IV"]["NLP-EXPANDED-prammed-postprocessed_translation"]
col_syn   = client["En2DeSyn"]["ED_deSynonyms"]

# Einzigartige convoIDs
convo_ids = col_trans.distinct("convoID", {"excludeGeneral": 0})
df_trans  = pd.DataFrame(convo_ids, columns=["convoID"])

# ICD-Titel aus MIMIC
rows = []
for doc in col_mimic.find(
    {"stay_id": {"$in": convo_ids}},
    {"stay_id":1, "json_data_used.ED-Diagnosis-prammed.icd_title":1, "_id":0}
):
    rows.append({
        "convoID": doc["stay_id"],
        "icd_title": doc.get("json_data_used", {})
                        .get("ED-Diagnosis-prammed", {})
                        .get("icd_title")
    })
df_mimic = pd.DataFrame(rows)

# Synonyme aus En2DeSyn
icd_list = df_mimic["icd_title"].dropna().unique().tolist()
syn_rows = []
for doc in col_syn.find({"icd_title": {"$in": icd_list}}, {"icd_title":1, "de_synonyms":1, "_id":0}):
    syns = doc.get("de_synonyms") or []
    syn_rows.append({"icd_title": doc["icd_title"], "syn_count": len(syns)})
df_syn = pd.DataFrame(syn_rows)

# Zusammenführen
df = (
    df_trans
    .merge(df_mimic, on="convoID", how="left")
    .merge(df_syn, on="icd_title", how="left")
)
df["syn_count"] = df["syn_count"].fillna(0).astype(int)

# Filter und CSV-Export
df_no_syn = df[df["syn_count"] == 0].copy()
output_path = "convo_no_synonyms.csv"
df_no_syn.to_csv(output_path, index=False)
print(f"Datei '{output_path}' gespeichert mit {len(df_no_syn)} Einträgen.")


Datei 'convo_no_synonyms.csv' gespeichert mit 39 Einträgen.


In [30]:
# Ziel: Erzeuge eine JSON-Lines Datei, in der für jeden fehlenden ICD-Titel
# das Feld `ed_id` leer bleibt, aber `icd_code` korrekt aus MIMIC-IV gezogen wird.

import os
import json
import pandas as pd
from pymongo import MongoClient

# 1) Pfade
cwd        = os.getcwd()
EXCEL_PATH = os.path.join(cwd, "missing_syn_manual_gpt.xlsx")
OUT_JSON   = os.path.join(cwd, "to_insert.json")

# 2) Excel einlesen
df = pd.read_excel(EXCEL_PATH)
print("Spalten:", df.columns.tolist())

# 3) Tatsächlichen Spaltennamen für die Synonymliste
SYN_COL = "de_synonyme"

# 4) Hilfsfunktion: Zelle → Python-Liste
def parse_syns(x):
    if isinstance(x, str):
        try:
            return json.loads(x)
        except:
            return [s.strip() for s in x.split(",") if s.strip()]
    elif isinstance(x, (list, tuple)):
        return list(x)
    else:
        return []

# 5) MongoDB-Verbindung für ICD-Code Lookup
client     = MongoClient("mongodb://localhost:27018/")
col_mimic  = client["MIMIC-IV"]["NLP-EXPANDED-prammed-postprocessed_translation"]

# 6) JSON-Dokumente sammeln
records = []
for _, row in df.iterrows():
    convo_id   = str(row["convoID"])
    icd_title  = row["icd_title"]
    de_syns    = parse_syns(row[SYN_COL])

    # ICD-Code aus MIMIC-IV ziehen
    mimic = col_mimic.find_one({"stay_id": convo_id})
    if mimic and "json_data_used" in mimic:
        diag     = mimic["json_data_used"].get("ED-Diagnosis-prammed", {})
        icd_code = diag.get("icd_code")
    else:
        icd_code = None

    # Dokument vorbereiten (ed_id leer lassen)
    doc = {
        "ed_id":       "",            # bleibt leer
        "icd_code":    icd_code,
        "icd_title":   icd_title,
        "eng_cuis":    [],            # leer
        "de_synonyms": de_syns
    }
    records.append(doc)

# 7) JSON-Lines schreiben
with open(OUT_JSON, "w", encoding="utf8") as f:
    for rec in records:
        f.write(json.dumps(rec, ensure_ascii=False) + "\n")

print(f"✅ {len(records)} Dokumente in JSON-Lines geschrieben nach:\n   {OUT_JSON}")


Spalten: ['convoID', 'icd_title', 'syn_count', 'de_synonyme']
✅ 39 Dokumente in JSON-Lines geschrieben nach:
   /Users/mosimacnew/Code/speechbrain-fix/myNotebooks/2_STT-Metriken/mwer/to_insert.json


In [31]:
import pandas as pd

# JSON-Lines als DataFrame laden
df = pd.read_json("to_insert.json", lines=True)
display(df)


,ed_id,icd_code,icd_title,eng_cuis,de_synonyms
0,,S02631A,"Oth fracture of head and neck of right femur, ...",[],[Fraktur des Kopfes und Halses des rechten Obe...
1,,S72091A,Oth intartic fracture of lower end of left rad...,[],[Intraartikuläre Fraktur des distalen Endes de...
2,,S0101XA,"Foreign body in left ear, initial encounter",[],"[Fremdkörper im linken Ohr, Erstbegegnung, Ohr..."
3,,3090,PHYSICAL RESTRAINTS STATUS,[],"[Status körperlicher Fixierungen, Körperliche ..."
4,,5789,Unspecified atrial fibrillation,[],"[Unspezifisches Vorhofflimmern, Vorhof Flimmer..."
5,,R109,ABDOMINAL PAIN OTHER SPECIED,[],"[Bauchschmerz, sonst näher bezeichnet, Bauch S..."
6,,S42001A,"Unsp fracture of T5-T6 vertebra, init for clos fx",[],[Nicht näher bezeichnete Fraktur der Wirbel T5...
7,,R109,ABDOMINAL PAIN OTHER SPECIED,[],"[Bauchschmerz, sonst näher bezeichnet, Bauch S..."
8,,25000,"Inhalant dependence, uncomplicated",[],"[Abhängigkeit von Inhalantien, Inhalantien Abh..."
9,,O9943,"CAD UNSPEC VESSEL, NATIVE OR GRAFT",[],"[Koronare Herzkrankheit, Gefäss nicht näher be..."


In [32]:
# Ziel: Lade die JSON-Lines mit deinen 39 Dokumenten und füge sie alle in die Collection ED_deSynonyms ein.

from pymongo import MongoClient
import json
import os

# 1) Pfade und Verbindungen
cwd       = os.getcwd()
JSON_PATH = os.path.join(cwd, "to_insert.json")

client  = MongoClient("mongodb://localhost:27018/")
db      = client["En2DeSyn"]
col     = db["ED_deSynonyms"]

# 2) JSON-Lines einlesen
docs = []
with open(JSON_PATH, "r", encoding="utf8") as f:
    for line in f:
        line = line.strip()
        if not line:
            continue
        docs.append(json.loads(line))

# 3) Einfügen in die Collection
if docs:
    result = col.insert_many(docs)
    print(f"Eingefügt {len(result.inserted_ids)} Dokumente in ED_deSynonyms.")
else:
    print("Keine Dokumente zum Einfügen gefunden.")


Eingefügt 39 Dokumente in ED_deSynonyms.


In [33]:
# Ziel: Für jede einzigartige convoID prüfen, welcher ICD-Titel zugeordnet ist und wie viele deutsche Synonyme dafür vorliegen.

from pymongo import MongoClient
import pandas as pd
from IPython.display import display

# 1. MongoDB-Verbindung und Collections
client    = MongoClient("mongodb://localhost:27018/")
col_trans = client["transcriptions"]["transcripts_denis"]
col_mimic = client["MIMIC-IV"]["NLP-EXPANDED-prammed-postprocessed_translation"]
col_syn   = client["En2DeSyn"]["ED_deSynonyms"]

# 2. Einzigartige convoIDs mit excludeGeneral=0
convo_ids = col_trans.distinct("convoID", {"excludeGeneral": 0})
df_trans  = pd.DataFrame(convo_ids, columns=["convoID"])

# 3. ICD-Titel aus MIMIC für diese convoIDs
mimic_cursor = col_mimic.find(
    {"stay_id": {"$in": convo_ids}},
    {"stay_id": 1, "json_data_used.ED-Diagnosis-prammed.icd_title": 1, "_id": 0}
)
rows = []
for doc in mimic_cursor:
    icd = (doc.get("json_data_used", {})
              .get("ED-Diagnosis-prammed", {})
              .get("icd_title"))
    rows.append({"convoID": doc["stay_id"], "icd_title": icd})
df_mimic = pd.DataFrame(rows)

# 4. Deutsche Synonyme und Zähler aus ED_deSynonyms
icd_list = df_mimic["icd_title"].dropna().unique().tolist()
syn_cursor = col_syn.find(
    {"icd_title": {"$in": icd_list}},
    {"icd_title": 1, "de_synonyms": 1, "_id": 0}
)
syn_rows = []
for doc in syn_cursor:
    syns = doc.get("de_synonyms") or []
    syn_rows.append({
        "icd_title": doc["icd_title"],
        "synonyms": syns,
        "syn_count": len(syns)
    })
df_syn = pd.DataFrame(syn_rows)

# 5. Zusammenführen und fehlende Einträge auffüllen
df = (
    df_trans
    .merge(df_mimic, on="convoID", how="left")
    .merge(df_syn, on="icd_title", how="left")
)
df["synonyms"] = df["synonyms"].apply(lambda x: x if isinstance(x, list) else [])
df["syn_count"] = df["syn_count"].fillna(0).astype(int)

# 6. Ergebnis anzeigen
display(df)

# Überblick
print(f"Anzahl eindeutiger convoIDs: {len(df)}")
print(f"davon mit ≥1 deutschem Synonym: {(df['syn_count'] > 0).sum()}")
print(f"davon ohne Synonyme:           {(df['syn_count'] == 0).sum()}")

,convoID,icd_title,synonyms,syn_count
0,30013825,"Oth fracture of head and neck of right femur, ...",[Fraktur des Kopfes und Halses des rechten Obe...,3
1,30114157,Oth intartic fracture of lower end of left rad...,[Intraartikuläre Fraktur des distalen Endes de...,3
2,30238398,"Foreign body in left ear, initial encounter","[Fremdkörper im linken Ohr, Erstbegegnung, Ohr...",3
3,30328724,Nausea,"[an uebelkeit leiden, an uebelkeit leidend, na...",8
4,30394981,MELENA,"[blut im stuhl, blutiger stuhl, blutstuhl]",3
...,...,...,...,...
104,39609911,"Other psychoactive substance abuse, uncomplicated","[Missbrauch anderer psychoaktiver Substanzen, ...",3
105,39637559,ABDOMINAL PAIN EPIGASTRIC,"[epigastralgie, epigastrischer schmerz, epigas...",4
106,39639914,DIAB HYPEROSM COMA IDDM,[Hyperosmolares Koma bei insulinpflichtigem Di...,3
107,39699150,LONG-TERM (CURRENT) USE OF INSULIN,"[Langzeitgebrauch von Insulin, Insulin Gebrauc...",3


Anzahl eindeutiger convoIDs: 109
davon mit ≥1 deutschem Synonym: 109
davon ohne Synonyme:           0
